# AIPI510 Fall 2024 Assignment # 2

## Team Details

**Team name:** Forever Loop

**Team members:** Santosh Ganesan, Haran Nallasivan

### CREATE
You have been provided with a sample .csv file. Create a SQLite database in Python using the data in this sample .csv file.

#### Imports

In [1]:
import pandas as pd
import sqlite3

#### Exploratory analysis of the data

In [2]:

df = pd.read_csv('data/tips.csv')
print(df.head())

   total_bill   tip     sex smoker  day    time  size
0       16.99  1.01  Female     No  Sun  Dinner     2
1       10.34  1.66    Male     No  Sun  Dinner     3
2       21.01  3.50    Male     No  Sun  Dinner     3
3       23.68  3.31    Male     No  Sun  Dinner     2
4       24.59  3.61  Female     No  Sun  Dinner     4


#### Create the SQLite database

In [3]:

with sqlite3.connect('tips.db') as conn:
    c = conn.cursor()
    c.execute('''CREATE TABLE tips (total_bill,tip,sex,smoker,day,time,size)''')
    conn.commit()

#### Verify connectivity to the database

In [4]:
with sqlite3.connect('tips.db') as conn:
    c = conn.cursor()
    c.execute('''SELECT * from tips;''')
    conn.commit()
    print(c.fetchall())

[]


#### Load data into the database

In [ ]:
with sqlite3.connect('tips.db') as conn:
    df = pd.read_csv('data/tips.csv')
    df.to_sql('tips', conn, if_exists='replace', index=False)
    conn.commit()


### READ
Answer the following questions (must show your work via SQL queries):
1. Retrieve the average tip percentage for each day of the week 


2. Find the maximum and minimum total bull amounts


3. Count the number of parties for each size


4. Retrieve the total bill and tip for parties of size 4 or more, where the tip percentage is greater than 15%



5. Retrieve the total bill, tip amount, and tip percentage for each combination of day and time, sorted by tip percentage in descending order

6. Find the average tip percentage for each combination of day, time, and smoker status


7. Retrieve the total bill, tip amount, and tip percentage for each sex, sorted by total bill in descending order, and limit the results to the top 5 records


8. Find the maximum and minimum tip percentage for each day and time combination, along with the corresponding total bill and tip amount


9. Retrieve the total bill, tip amount, and tip percentage for parties of size 4 or more, where the tip percentage is greater than 15%, and the total bill is between $50 and $100


10. Find the average tip percentage for each combination of day, time, and smoker status, but only include combinations with more than 5 records


Come up with your own SQL queries for the table. You must show 5 additional queries beyond the 10 outlined above. 

**After you have performed the above queries:**



### UPDATE
It was determined that there was an error in the database. Please update the record that corresponds to id=10 and set smoker to Yes. 




### DELETE
Delete records from the database that have a total bill that is less than $10.